In [150]:
from sklearn.tree import DecisionTreeClassifier
from interpret.glassbox import ExplainableBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB


clfs = {
    "CART": DecisionTreeClassifier(random_state=1234),
    "EBM": ExplainableBoostingClassifier(),
    "LR": LogisticRegression(random_state=1234),
    "GNB": GaussianNB()
    
}

In [166]:
## add k fold split

In [151]:
import pandas as pd
import numpy as np

import helper
import importlib
importlib.reload(helper)

from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline

In [152]:
# datasets = ['breast', 'campus', 'churn', 'climate',
#             'compas', 'diabetes', 'german', 'heart',
#             'stroke', 'student', 'thoracic', 'water']
#fix thoracic dataset

datasets = ['breast', 'campus', 'churn', 'climate',
            'compas', 'diabetes', 'german', 'heart',
            'stroke', 'student', 'water']


In [153]:
# datasets = ['thoracic']

In [154]:
X=pd.read_csv(f"datasets/cleaned/{datasets[4]}_X.csv")
X = X.drop("Unnamed: 0", axis=1)
y = pd.read_csv(f"datasets/cleaned/{datasets[4]}_y.csv")
y = y.drop("Unnamed: 0", axis=1)

features_types_df = pd.read_csv(f"datasets/cleaned/datatypes/{datasets[4]}.csv")

feature_inidices = list(map(int, list(features_types_df)))
features_names = list(features_types_df.T[0])
features_types = list(map(int, list(features_types_df.T[1])))

preprocess = helper.select_preprocessing_for_many_feat(feature_inidices, features_types, features_names)

In [155]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=1410)

In [156]:
from sklearn.model_selection import StratifiedShuffleSplit
subsample = StratifiedShuffleSplit(n_splits = 8, train_size= 0.5, test_size=0.5, random_state=1610)

In [157]:
subset_results=[]
subset_acc=[]
ground_true_labels=[]
for sub_idx, (train_idx, test) in enumerate(subsample.split(X_train, y_train)):
  X, y = X_train.iloc[train_idx], y_train.iloc[train_idx]
  X_t, y_t = X_train.iloc[test], y_train.iloc[test]

  heart_ebm = make_pipeline(
      preprocess, 
      GaussianNB()
  )
  heart_ebm.fit(X, y)
  heart_ebm_preds = heart_ebm.predict(X_t)
  # accuracy_scr = accuracy_score(y_true = y_t, y_pred = heart_ebm_preds)
  subset_results.append(heart_ebm_preds)
  ground_true_labels.append(y_t)
  # subset_acc.append(accuracy_scr)

/home/bart/anaconda3/envs/mainenv/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/bart/anaconda3/envs/mainenv/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/bart/anaconda3/envs/mainenv/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/bart/anaconda3/envs/mainenv/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed w

In [158]:
ground_true_labels[0]

,Misdemeanor
1343,0
1541,1
4418,0
4560,0
2482,0
...,...
2181,0
5067,1
2341,1
66,1


In [159]:
subset_results[0]

array([1, 1, 1, ..., 0, 0, 0])

In [160]:
variance = np.mean(np.var(np.array(subset_results), axis=0))

In [161]:
variance

0.22585660503240565

In [162]:
avg_test_y_pred = np.mean(np.array(subset_results), axis=0)

In [163]:
avg_test_y_pred

array([0.625, 0.5  , 0.375, ..., 0.5  , 0.5  , 0.625])

In [164]:
bias = np.mean((avg_test_y_pred - ground_true_labels) ** 2)

In [165]:
bias

0.2862953655193093